In [1]:
import os

import numpy as np
import torch

from Bio.PDB import PDBList
from Bio.PDB.MMCIFParser import MMCIFParser
import re

#phi and psi
from Bio.PDB import PICIO, PDBIO
from Bio import PDB
from typing import TypedDict, Dict, Tuple


from Bio.PDB import PDBList
import pandas as pd

from transformers import BertModel, BertTokenizer
import re


import pickle

/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


put the first file here

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:

tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False ) # change model and tokenizer to t5
model_embedd = BertModel.from_pretrained("Rostlab/prot_bert")
model_embedd = model_embedd.to(device)


You need to upload a huge folder of all the data here. The pdb files folder in directory path. In target path upload the inputs folder

In [4]:

#def is_chain_continuous(chain): # this function is not necessary, just checks if the chains are continuous
    #prev_resnum = None
    #for residue in chain:
    #    if prev_resnum is not None and residue.get_id()[1] != prev_resnum + 1:
    #        return False
    #    prev_resnum = residue.get_id()[1]
    #return True

def extract_input(pdb_file):
    #parser = PDBParser()
    #structure = parser.get_structure("protein", pdb_file)
    #model0 = structure[0]

    #file_model = file_path.replace(".cif","") # loading file
    #pdbl = PDBList()
    cif_parser = MMCIFParser()
    structure = cif_parser.get_structure("protein", pdb_file) # getting structure
    model0 = structure[0]


    mapping_file = os.path.join('pdb_id_mapping.csv') # this is to access the dictionary keys, which are the same as the file name, once the jeys are accessed we can find which chain to take
    pdb_id_map = pd.read_csv(mapping_file, header=None, index_col=0).squeeze().to_dict()

    #extracting the file name as it contains the id
    filename = os.path.basename(pdb_file)
    pdb_id = filename[:4] # the first 4 letters of the file name is the dictionary key,whose value is the chain to be used
    pdb_id = pdb_id
    #print(pdb_id)

    full_pdb_id = pdb_id_map.get(pdb_id, None) #pdb id with the correct chain


    #chain_A = model0[full_pdb_id[-1]]  # and we get chain A , the last letter of the id is the chain
    #print("full pd ib:", full_pdb_id)

    # this is a very common need in bioinformatics of proteins
    d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
    'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
    'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
    'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

    #sequence = []
    #for residue in chain_A:
        # for simplicity we can use X for heteroatoms (ions and water)
    #    sequence.append(d3to1.get(residue.get_resname(), 'X'))  #converts water and ions to X
    #print(len(sequence))
    #print(''.join(sequence))
    #seq_example =  ' '.join(sequence)
    #return seq_example

#def extract_coordinates(pdb_file):
    structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths
    if full_pdb_id[-1] == 'X':
            print("X")
            chain_index = 0
    elif full_pdb_id[-1].isdigit():
       # If the last character is a digit
       chain_index = int(full_pdb_id[-1]) - 1 
       print("digit:", chain_index )
    else:
        # If the last character is not a digit (assuming it's a letter)
        chain_index = ord(full_pdb_id[-1]) - ord('A')
        print("letter: ", chain_index )
    
    #chain_index = ord(full_pdb_id[-1]) - ord('A') 
    #print(full_pdb_id)
    #print(full_pdb_id[-1]) # Calculate index based on 'A' = 0, 'B' = 1, etc.
    #print(type(full_pdb_id[-1]))
    #print(chain_index)
    #print(len(list(model0.get_chains())))

    #creating a dictionary to collect chain dis and turn them into indices
    chain_id_to_num = {}
    for num, chain in enumerate(model0.get_chains()):
        chain_id_to_num[chain.id] = num
    chain:PDB.Chain.Chain = list(model0.get_chains())[chain_id_to_num.get(full_pdb_id[-1])]#iterator of chains, turns it into list, [0] first chain

    ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object

    d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

    cnt = 1
    phi_angles = {}
    phi_angles_list = []
    psi_angles = {}
    psi_angles_list = []

    for key in d:
      if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':
          psi_angles[key] = d[key].angle
          psi_angles_list.append(d[key].angle)
      elif key[0].akl[3] == 'C' and key[1].akl[3] == 'N' and key[2].akl[3] == 'CA' and key[3].akl[3] == 'C':
          phi_angles[key] = d[key].angle
          phi_angles_list.append(d[key].angle)


    structure.internal_to_atom_coordinates(verbose = False)#before it was true, you can change this back if theres any
    io = PDBIO() #this is to write a pdb file again
    io.set_structure(structure)#set structure, the structure you wan tin the pdb file
    phi_angles_list.append(0)
    psi_angles_list.append(0)

    phi = np.asarray(phi_angles_list,dtype=np.float32)
    psi = np.asarray(psi_angles_list,dtype=np.float32)
    angles = np.vstack((psi,phi))
    angle_tensor = torch.tensor(angles, dtype=torch.float32).to(device)


    #seq = re.sub(r"[UZOB]", "X", seq_example)

    #encoded_input = tokenizer.encode(sequence, return_tensors='pt')  #here the tokenizer you use



    #Make predictions
    #with torch.no_grad():
    #    outputs = model(input_ids=encoded_input)

    #embeddings = outputs.last_hidden_state
    #print(f"Shape of embeddings: {embeddings.shape}")
    #outputs = model(**encoded_input)
    
    #embeddings = outputs.last_hidden_state


    #print('N:', N,D)
    return angle_tensor #you may not need to extract sequence info for each file

In [5]:

directory_path = 'small proteins'
target_path = 'new_training/'
embeddings_file = os.path.join(target_path, 'embeddings.pt')
angles_file = os.path.join(target_path, 'angles.pt')
sequences_file = os.path.join(target_path, 'sequences.pkl') # you may not need this

# Load existing tensors if they exist
if os.path.exists(embeddings_file) and os.path.exists(angles_file): # not imp for your case, checking if we already have files from previous uncrashed runtimes
    embeddings = torch.load(embeddings_file)
    angles = torch.load(angles_file)

    with open(sequences_file, 'rb') as f:
        sequences = pickle.load(f)

    
    #print(embeddings.size(0))
    start_index = embeddings.size(0)


else:
    #notice i have padded embeddings so they are all the same size. i am not sure if that will cause issues
    # as im making everything zero, and later filling it, automatically the unfilled parts are padded with zeros
    embeddings = torch.zeros(0, 131, 1024).to(device) # embeddings max size is 130, because you have astart and an end token

    angles = torch.zeros(0, 2, 131).to(device)
    sequences = []
    start_index = 0

# Number of files to process in this run
num_files_to_process = 607 #then 90

# Iterate over the files to process
print("start index: ", start_index)
i = start_index

def pad_sequence(sequence, target_length=129, pad_char="X"): # may not need this as im using a padding in the tokenizer itself
    padding_length = target_length - len(sequence)
    padding = [pad_char] * padding_length
    return sequence + padding

# Function to get last modified time of a file
def get_file_modified_time(file_path):
    return os.path.getmtime(file_path)

# Get list of files sorted by last modified time
file_list = sorted(os.listdir(directory_path), key=lambda x: get_file_modified_time(os.path.join(directory_path, x)), reverse=False)
#file_list.sort()  # not very neccesary

print(file_list)

for filename in file_list[start_index:start_index + num_files_to_process]:
    file_path = os.path.join(directory_path, filename)
    #print(file_path)
    if i >= start_index + num_files_to_process:
        break


    if os.path.isfile(file_path):
        print(i)
        angle= extract_input(file_path) # the key part that extracts all the information into a file
        angle_length = min(angle.size(1), 131)
        if i >= angles.size(0):

            angles = torch.cat([angles, torch.zeros(1, 2, 131).to(device)], dim=0)
        
        angles[i, :, :angle_length] = angle[:, :]

        i += 1
        print(i)


# Save the updated tensors back to the files

torch.save(angles, angles_file)


# Path to the folder you want to download
#folder_path = '/content/map'
# Name of the output zip file
#zip_name = 'inputs.zip' # downloads file with info

# Zip the folder
#shutil.make_archive(zip_name.replace('.zip', ''), 'zip', target_path)

# Download the zip file
#files.download(zip_name)

start index:  0
['1e0z.cif', '1ry4.cif', '1ug7.cif', '1whn.cif', '2aya.cif', '2d87.cif', '2db5.cif', '2dkp.cif', '2ebk.cif', '2ett.cif', '2fm4.cif', '2i4k.cif', '2jq5.cif', '2k0r.cif', '2k5t.cif', '2lr4.cif', '2ls0.cif', '2lvl.cif', '2mah.cif', '2ncn.cif', '2rlo.cif', '2rv8.cif', '2ytu.cif', '4ar0.cif', '5aiw.cif', '5h7u.cif', '5n9q.cif', '8e22.cif', '1cmo.cif', '1ddf.cif', '1eal.cif', '1pqn.cif', '1qkl.cif', '1uey.cif', '1w2q.cif', '1wh4.cif', '1wjr.cif', '1x5m.cif', '1zu1.cif', '2cso.cif', '2dmk.cif', '2ee7.cif', '2l1l.cif', '2loe.cif', '2mf7.cif', '2mpl.cif', '2nbo.cif', '2tmp.cif', '4uei.cif', '5m1u.cif', '7xrw.cif', '8oil.cif', '1rsf.cif', '1v5s.cif', '1x5e.cif', '1zo0.cif', '2a4h.cif', '2cr4.cif', '2dav.cif', '2if1.cif', '2jp2.cif', '2jvn.cif', '2k2v.cif', '2l3g.cif', '2l57.cif', '2lbc.cif', '2lln.cif', '2m26.cif', '3msp.cif', '7f7n.cif', '7knv.cif', '8sqx.cif', '1buq.cif', '1bw3.cif', '1fr0.cif', '1gio.cif', '1mh6.cif', '1psy.cif', '1uen.cif', '1wyj.cif', '2ai6.cif', '2dhj.cif',

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f768fd727a0>>
Traceback (most recent call last):
  File "/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


letter:  0
23
23
letter:  0
24
24
letter:  0
25
25
letter:  0
26
26
letter:  0
27
27
X
28
28
letter:  0
29
29
letter:  0
30
30
letter:  0
31
31
letter:  0
32
32
letter:  0
33
33
letter:  0
34
34
letter:  0
35
35
letter:  0
36
36
letter:  0
37
37
letter:  0
38
38
letter:  0
39
39
letter:  0
40
40
letter:  0
41
41
letter:  0
42
42
letter:  1
43
43
letter:  0
44
44
letter:  0
45
45
letter:  0
46
46
letter:  0
47
47
letter:  0
48
48
letter:  0
49
49
letter:  0
50
50
letter:  0
51
51
letter:  0
52
52
letter:  0
53
53
letter:  0
54
54
letter:  0
55
55
letter:  0
56
56
letter:  0
57
57
letter:  0
58
58
letter:  0
59
59
letter:  0
60
60
letter:  0
61
61
letter:  0
62
62
letter:  0
63
63
letter:  0
64
64
letter:  0
65
65
letter:  0
66
66
letter:  0
67
67
letter:  0
68
68
letter:  0
69
69
letter:  0
70
70
letter:  0
71
71
letter:  0
72
72


/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3746.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3802.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 7604.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 7660.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 11462.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/

letter:  0
73
73
letter:  0
74
74
letter:  0
75
75
letter:  0
76
76
letter:  0
77
77
letter:  0
78
78
letter:  0
79
79
letter:  0
80
80
letter:  0
81
81
letter:  0
82
82
letter:  0
83
83
letter:  0
84
84
letter:  0
85
85
letter:  0
86
86
letter:  0
87
87
letter:  0
88
88
letter:  0
89
89
letter:  0
90
90
letter:  0
91
91
letter:  0
92
92
letter:  1
93
93
letter:  0
94
94
letter:  0
95
95
letter:  0
96
96
letter:  0
97
97
letter:  0
98
98
letter:  0
99
99
letter:  0
100
100
letter:  0
101
101
letter:  0
102
102
letter:  0
103
103
letter:  0
104
104
letter:  0
105
105
letter:  0
106
106
letter:  0
107
107
letter:  0
108
108
letter:  0
109
109
letter:  0
110
110
letter:  0
111
111
letter:  0
112
112
letter:  0
113
113
letter:  0
114
114
letter:  0
115
115
letter:  0
116
116
letter:  0
117
117
letter:  0
118
118
letter:  0
119
119
letter:  0
120
120
letter:  0
121
121
letter:  0
122
122
letter:  0
123
123
letter:  0
124
124
letter:  0
125
125
letter:  0
126
126
letter:  0
127
127
letter:  

/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 17056.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 17061.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 17066.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 17071.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 34128.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/

letter:  3
152
152
letter:  0
153
153
letter:  0
154
154
letter:  0
155
155
letter:  0
156
156
letter:  0
157
157
letter:  0
158
158
letter:  0
159
159
letter:  0
160
160
letter:  0
161
161
letter:  0
162
162
letter:  0
163
163
letter:  0
164
164
letter:  0
165
165
letter:  0
166
166
letter:  0
167
167
letter:  0
168
168
letter:  0
169
169
letter:  0
170
170
letter:  0
171
171
letter:  0
172
172
letter:  0
173
173
letter:  0
174
174
letter:  0
175
175
letter:  0
176
176
letter:  0
177
177
letter:  0
178
178
letter:  0
179
179
letter:  0
180
180
letter:  0
181
181
letter:  0
182
182
letter:  0
183
183
letter:  0
184
184
letter:  0
185
185
letter:  0
186
186
letter:  0
187
187
letter:  0
188
188
letter:  0
189
189
letter:  0
190
190
letter:  0
191
191
letter:  0
192
192
letter:  0
193
193
letter:  0
194
194
letter:  0
195
195
letter:  0
196
196
letter:  0
197
197
letter:  0
198
198
letter:  0
199
199
letter:  0
200
200
letter:  0
201
201
letter:  0
202
202
letter:  0
203
203
letter:  0
2

/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 5166.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 10387.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 15608.
  warnings.warn(
/home/mihriban/gf_transformer/.venv/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 20829.
  warnings.warn(


letter:  0
374
374
letter:  0
375
375
letter:  0
376
376
letter:  0
377
377
letter:  0
378
378
letter:  0
379
379
letter:  0
380
380
letter:  0
381
381
letter:  0
382
382
letter:  0
383
383
letter:  0
384
384
letter:  0
385
385
letter:  0
386
386
letter:  0
387
387
letter:  0
388
388
letter:  0
389
389
X
390
390
letter:  0
391
391
letter:  0
392
392
letter:  0
393
393
letter:  0
394
394
letter:  0
395
395
letter:  0
396
396
letter:  0
397
397
letter:  0
398
398
letter:  0
399
399
letter:  0
400
400
letter:  0
401
401
letter:  0
402
402
letter:  0
403
403
letter:  0
404
404
letter:  0
405
405
letter:  0
406
406
letter:  0
407
407
letter:  0
408
408
letter:  0
409
409
letter:  0
410
410
letter:  0
411
411
letter:  0
412
412
letter:  0
413
413
letter:  0
414
414
letter:  0
415
415
letter:  0
416
416
letter:  0
417
417
letter:  0
418
418
letter:  0
419
419
letter:  0
420
420
letter:  0
421
421
letter:  0
422
422
letter:  0
423
423
letter:  0
424
424
letter:  0
425
425
letter:  0
426
426
le

In [ ]:
print(np.shape(embedding[:,:,:]))
print(np.shape(embeddings[:,:,:]))

print(np.shape(angle))
print(np.shape(angles))

In [ ]:
angles = torch.load('angles.pt')
phi_angles = angles[:,0,:]
psi_angles = angles[:,1,:]

print(angles.size())
print(phi_angles.size())
print(angles[15,:,:])